# VisualOdometrySLAM

In [108]:
import random

import cv2
import numpy as np
import pandas as pd

def seed_all(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    cv2.setRNGSeed(seed)

In [109]:
SEED = 42
seed_all(SEED)

In [110]:
from pathlib import Path

DATA_PATH = Path('../data/')
DATA_PATH.mkdir(parents=True, exist_ok=True)

DATA_PATH_VIDEO = DATA_PATH /Path('visual_odometry/')
DATA_PATH_VIDEO.mkdir(parents=True, exist_ok=True)

DATA_PATH_OUTPUT = DATA_PATH / Path('output_data/')
DATA_PATH_OUTPUT.mkdir(parents=True, exist_ok=True)

DATA_PATH_SAVE_MODELS = DATA_PATH / Path('models/')
DATA_PATH_SAVE_MODELS.mkdir(parents=True, exist_ok=True)

DATA_IMGS = Path('../imgs')
DATA_IMGS.mkdir(parents=True, exist_ok=True)

In [111]:
import sys
import os

project_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_path)

## Общие методы

| Параметр                  | Где в YAML                     | Что означает                                  | Как определить / откуда берётся                 |
| ------------------------- | ------------------------------ | --------------------------------------------- | ----------------------------------------------- |
| `fx`, `fy`                | `camera_matrix.data[0]`, `[4]` | Фокусное расстояние по осям X и Y             | Из калибровки (например, `cv2.calibrateCamera`) |
| `cx`, `cy`                | `camera_matrix.data[2]`, `[5]` | Координаты главной точки (оптический центр)   | Центр кадра или из калибровки                   |
| `camera_matrix`           | `camera_matrix.data`           | Внутренняя матрица 3×3                        | Результат калибровки                            |
| `distortion_model`        | `distortion_model`             | Модель дисторсии (`plumb_bob`, `equidistant`) | Выбирается при калибровке, по типу объектива    |
| `distortion_coefficients` | `distortion_coefficients.data` | Коэффициенты искажений (k1, k2, p1, p2, k3)   | Выход `cv2.calibrateCamera`, `Kalibr`, ROS      |
| `image_width` / `height`  | `image_width`, `image_height`  | Размер изображения (в пикселях)               | Известно по сенсору или из примера калибровки   |
| `camera_name`             | `camera_name`                  | Название камеры                               | Задаётся вручную (опционально)                  |


In [112]:
import yaml
import numpy as np
from pathlib import Path
from typing import Union, Tuple, Optional

def load_calibration(
    calibration_file: Union[str, Path]
) -> Tuple[np.ndarray, Optional[np.ndarray], Optional[str]]:
    """
    Загрузка параметров калибровки камеры из YAML-файла в формате ROS/OpenCV.

    :param calibration_file: Путь к YAML-файлу с калибровкой (ROS-стиль)
    :return: Кортеж (K, distCoeffs, distortion_model), где:
             K — внутренняя матрица камеры (3×3),
             distCoeffs — вектор коэффициентов дисторсии (или None),
             distortion_model — строка с названием модели искажений (или None)
    """
    calibration_file = Path(calibration_file)
    if not calibration_file.exists():
        raise FileNotFoundError(f"Файл калибровки не найден: {calibration_file.resolve()}")

    with calibration_file.open('r') as f:
        data = yaml.safe_load(f)

    # Чтение матрицы камеры (3×3)
    K_data = data["camera_matrix"]["data"]
    if len(K_data) != 9:
        raise ValueError("camera_matrix должна содержать 9 элементов.")
    K = np.array(K_data, dtype=np.float64).reshape(3, 3)

    # Чтение коэффициентов дисторсии (опционально)
    dist_data = data.get("distortion_coefficients", {}).get("data", [])
    distCoeffs = np.array(dist_data, dtype=np.float64) if dist_data else None

    # Чтение модели искажений (опционально)
    model = data.get("distortion_model", None)

    return K, distCoeffs, model


In [113]:
import cv2
import numpy as np
from pathlib import Path
from typing import Union, List


def load_video_frames(video_path: Union[str, Path], target_fps: float) -> List[np.ndarray]:
    """
    Извлекает кадры из видео с заданной частотой и переводит их в оттенки серого.

    :param video_path: путь к видеофайлу
    :param target_fps: целевая частота обработки кадров
    :return: список кадров (в оттенках серого)
    """
    video_path = Path(video_path)
    if not video_path.exists():
        raise FileNotFoundError(f'Видео не найдено по пути: {video_path.resolve()}')

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise IOError(f'Не удалось открыть видеофайл: {video_path}')

    video_fps = cap.get(cv2.CAP_PROP_FPS)
    frame_interval = int(video_fps / target_fps)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration_sec = total_frames / video_fps
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    print(f"Информация о видео:")
    print(f"- Имя файла: {video_path.name}")
    print(f"- Размер кадра: {frame_width} x {frame_height}")
    print(f"- Оригинальный FPS: {video_fps:.2f}")
    print(f"- Общее число кадров: {total_frames}")
    print(f"- Длительность видео: {duration_sec:.2f} секунд")
    print(f"- Целевая частота обработки: {target_fps} FPS")
    print(f"- Будет обрабатываться каждый {frame_interval}-й кадр")

    processed_frames = []
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_idx % frame_interval == 0:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            processed_frames.append(gray)

        frame_idx += 1

    cap.release()
    print(f"Обработано кадров: {len(processed_frames)} при целевом FPS: {target_fps}")
    return processed_frames


## ORB params

| Параметр        | Тип     | Значение по умолчанию  | Описание и рекомендации                                                                                                         |
| --------------- | ------- | ---------------------- | ------------------------------------------------------------------------------------------------------------------------------- |
| `nfeatures`     | `int`   | 500                    | Максимальное количество фич, которые ORB попытается найти. <br>🔧 Увеличь до 3000–5000 для богатой сцены или сеточной детекции. |
| `scaleFactor`   | `float` | 1.2                    | Масштаб между уровнями пирамиды. <br>Меньше — больше перекрытий, лучше для масштабных сцен, но медленнее.                       |
| `nlevels`       | `int`   | 8                      | Кол-во уровней в пирамиде изображений. <br>🔧 Увеличь до 12–16, если ожидается масштабный диапазон объектов.                    |
| `edgeThreshold` | `int`   | 31                     | Отступ от границ изображения. <br>🔧 Уменьши до 10–15, если хочешь находить фичи у краёв.                                       |
| `firstLevel`    | `int`   | 0                      | Начальный уровень пирамиды (обычно 0). Почти никогда не меняется.                                                               |
| `WTA_K`         | `int`   | 2                      | Метод генерации BRIEF-дескриптора: 2 = быстрее, 3 или 4 = точнее. <br>Используй 2, если нет проблем с точностью.                |
| `scoreType`     | `int`   | `cv2.ORB_HARRIS_SCORE` | Выбор способа ранжирования углов: `cv2.ORB_HARRIS_SCORE` или `cv2.ORB_FAST_SCORE`. <br>`HARRIS` точнее, `FAST` — быстрее.       |
| `patchSize`     | `int`   | 31                     | Размер области вокруг фичи для дескриптора. <br>🔧 31 — стандарт, можно увеличить до 49, если объект крупный.                   |
| `fastThreshold` | `int`   | 20                     | Порог FAST-угла: чем меньше — тем больше фич. <br>🔧 Снизь до 5–10, если ORB пропускает фичи.                                   |


In [ ]:
from pathlib import Path
from typing import Union, List, Optional

import cv2
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt


class VisualOdometry:
    def __init__(self, 
                 video_path: Union[str, Path],
                 calibration_file: Union[str, Path],                   
                 **params) -> None:
        """
        Инициализация пайплайна визуальной одометрии.

        Загружает калибровку камеры и видео, а также позволяет задать параметры ORB-детектора через `params`.

        :param video_path: путь к видеофайлу (например, .mp4)
        :param calibration_file: путь к YAML-файлу с калибровкой (формат OpenCV / ROS)
        :param target_fps: частота обработки кадров
        :param params: дополнительные параметры, передаваемые в ORB.create(), например:
                       nfeatures=3000, edgeThreshold=10, fastThreshold=5 и т.п.
        """
        self.params: dict = params
        
        self.K: np.ndarray
        self.distCoeffs: Optional[np.ndarray]
        self.distortion_model: Optional[str]

        self.K, self.distCoeffs, self.distortion_model = load_calibration(calibration_file)
        target_fps = self.params.get("target_fps", 2)
        self.frames: List[np.ndarray] = load_video_frames(video_path, target_fps)

        # список ORB keypoints
        self.keypoints: List = []  
        # список сопоставлений            
        self.matches: List = []   
        # список 3D-точек (триангуляция)
        self.triangulated_points: List[np.ndarray] = []  
        # список (R, t)   
        self.poses: List = []              

        self.orb = self.create_orb()
        self.matcher = self.create_matcher()

    def clear_data(self):
        """
        Очищает все внутренние списки и данные.
        Используется для перезапуска пайплайна.
        """
        self.keypoints.clear()
        self.matches.clear()
        self.poses.clear()
        self.triangulated_points.clear()

    def create_matcher(self) -> cv2.DescriptorMatcher:
        """
        Создаёт и возвращает подходящий матчер дескрипторов для ORB.
        :return: объект cv2.DescriptorMatcher
        """
        FLANN_INDEX_LSH = 6
        index_params = dict(
            algorithm=FLANN_INDEX_LSH,
            table_number=self.params.get("flann_table_number", 6),
            key_size=self.params.get("flann_key_size", 12), 
            multi_probe_level=self.params.get("flann_multi_probe_level", 1)
        )
        search_params = dict(
            checks=self.params.get("flann_checks", 50)
        )
        return cv2.FlannBasedMatcher(index_params, search_params)
    
    # def create_matcher(self):
    #     return cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    
    def create_orb(self) -> cv2.ORB:
        """
        Создаёт ORB-детектор с параметрами из конструктора.
        :return: объект cv2.ORB
        """
        orb_params = {
            "nfeatures": self.params.get("nfeatures", 1000),
            "scaleFactor": self.params.get("scaleFactor", 1.2),
            "nlevels": self.params.get("nlevels", 8),
            "edgeThreshold": self.params.get("edgeThreshold", 31),
            "fastThreshold": self.params.get("fastThreshold", 20)
        }
        return cv2.ORB.create(**orb_params)

    def filter_by_ratio_test(self, knn_matches, kp1, kp2, ratio_thresh):
        good_matches = []
        pts1, pts2 = [], []

        for pair in knn_matches:
            if len(pair) < 2:
                continue
            m, n = pair
            if m.distance < ratio_thresh * n.distance:
                good_matches.append(m)
                pts1.append(kp1[m.queryIdx].pt)
                pts2.append(kp2[m.trainIdx].pt)

        pts1 = np.float32(pts1)
        pts2 = np.float32(pts2)
        return good_matches, pts1, pts2

    def filter_by_max_distance(self, matches, pts1, pts2, max_dist):
        filtered_matches = []
        filtered_pts1, filtered_pts2 = [], []

        for i, m in enumerate(matches):
            if m.distance < max_dist:
                filtered_matches.append(m)
                filtered_pts1.append(pts1[i])
                filtered_pts2.append(pts2[i])

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)
    
    def filter_unique_matches(self, matches, pts1, pts2):
        seen_trainIdx = set()
        unique_matches = []
        unique_pts1, unique_pts2 = [], []

        for i, m in enumerate(matches):
            if m.trainIdx not in seen_trainIdx:
                seen_trainIdx.add(m.trainIdx)
                unique_matches.append(m)
                unique_pts1.append(pts1[i])
                unique_pts2.append(pts2[i])

        return unique_matches, np.float32(unique_pts1), np.float32(unique_pts2)
    
    def filter_by_ransac_fundamental(self, matches, pts1, pts2):
        if len(matches) < 8:
            return [], np.array([]), np.array([])

        F, mask = cv2.findFundamentalMat(pts1, pts2, method=cv2.FM_RANSAC, ransacReprojThreshold=1.0)

        inlier_matches = []
        inlier_pts1, inlier_pts2 = [], []

        for i, m in enumerate(matches):
            if mask[i]:
                inlier_matches.append(m)
                inlier_pts1.append(pts1[i])
                inlier_pts2.append(pts2[i])

        return inlier_matches, np.float32(inlier_pts1), np.float32(inlier_pts2)
    
    def filter_by_exclusion_mask(self, matches, pts1, pts2):
        exclude_regions = self.params.get("mask_exclude_regions", [])
        if not exclude_regions:
            return matches, pts1, pts2

        filtered_matches = []
        filtered_pts1 = []
        filtered_pts2 = []

        for i, m in enumerate(matches):
            x1, y1 = pts1[i]
            x2, y2 = pts2[i]

            excluded = False
            for (xmin, ymin, xmax, ymax) in exclude_regions:
                if (xmin <= x1 <= xmax and ymin <= y1 <= ymax) or \
                (xmin <= x2 <= xmax and ymin <= y2 <= ymax):
                    excluded = True
                    break

            if not excluded:
                filtered_matches.append(m)
                filtered_pts1.append((x1, y1))
                filtered_pts2.append((x2, y2))

        return filtered_matches, np.float32(filtered_pts1), np.float32(filtered_pts2)

    def filter_mutual_matches(
        self,
        des1: np.ndarray,
        des2: np.ndarray,
        kp1: List[cv2.KeyPoint],
        kp2: List[cv2.KeyPoint],
        matches: List[cv2.DMatch],
        pts1: np.ndarray,
        pts2: np.ndarray
    ) -> Tuple[List[cv2.DMatch], np.ndarray, np.ndarray]:
        """
        Оставляет только взаимные совпадения (A→B и B→A).
        """
        reverse_matches = self.matcher.match(des2, des1)
        mutual_matches = []
        mutual_pts1, mutual_pts2 = [], []
        reverse_dict = {(m.trainIdx, m.queryIdx) for m in reverse_matches}
        for i, m in enumerate(matches):
            if (m.queryIdx, m.trainIdx) in reverse_dict:
                mutual_matches.append(m)
                mutual_pts1.append(pts1[i])
                mutual_pts2.append(pts2[i])
        return mutual_matches, np.float32(mutual_pts1), np.float32(mutual_pts2)
    
    def get_matches(self, img1: np.ndarray, img2: np.ndarray) -> Tuple:
        kp1, des1 = self.orb.detectAndCompute(img1, None)
        kp2, des2 = self.orb.detectAndCompute(img2, None)

        if des1 is None or des2 is None:
            return [], [], [], np.array([]), np.array([])

        knn_matches = self.matcher.knnMatch(des1, des2, k=2)

        # 1. Lowe's ratio test
        ratio_thresh = self.params.get("ratio_thresh", 0.6)
        good_matches, pts1, pts2 = self.filter_by_ratio_test(knn_matches, kp1, kp2, ratio_thresh)

        # 2. Максимальная дистанция
        max_dist = self.params.get("max_match_distance", 60)
        good_matches, pts1, pts2 = self.filter_by_max_distance(good_matches, pts1, pts2, max_dist)

        # 3. Удаление повторов
        good_matches, pts1, pts2 = self.filter_unique_matches(good_matches, pts1, pts2)

        # 4. RANSAC-фильтрация
        good_matches, pts1, pts2 = self.filter_by_ransac_fundamental(good_matches, pts1, pts2)
        
        # 5. Исключение по маске
        good_matches, pts1, pts2 = self.filter_by_exclusion_mask(good_matches, pts1, pts2)

        # 6. Взаимные совпадения
        if self.params.get("use_mutual_check", True):
            good_matches, pts1, pts2 = self.filter_mutual_matches(
                des1, des2, kp1, kp2, good_matches, pts1, pts2
            )
    
        return kp1, kp2, good_matches, pts1, pts2
    
    def estimate_pose_opencv(self, pts1: np.ndarray, pts2: np.ndarray) -> Optional[Tuple[np.ndarray, np.ndarray]]:
        """
        Упрощённая оценка движения камеры на основе встроенной функции OpenCV.

        :param pts1: координаты точек в первом изображении (Nx2)
        :param pts2: координаты точек во втором изображении (Nx2)
        :return: кортеж (R, t) — матрица поворота и вектор смещения; либо None при ошибке
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            print("Недостаточно точек для оценки движения.")
            return None
        E, mask = cv2.findEssentialMat(
            pts1, pts2, self.K, 
            method=cv2.RANSAC, 
            prob=0.999, 
            threshold=1.0
        )
        if E is None or mask is None or mask.sum() < 8:
            print("Не удалось вычислить матрицу Essential.")
            return None
        _, R, t, pose_mask = cv2.recoverPose(E, pts1, pts2, self.K, mask=mask)
        return R, t
    
    @staticmethod
    def _form_transformation_matrix(R: np.ndarray, t: np.ndarray) -> np.ndarray:
        """
        Формирует 4×4 однородную матрицу трансформации из R и t.

        :param R: матрица поворота 3×3
        :param t: вектор смещения (3,)
        :return: 4×4 матрица T
        """
        T = np.eye(4, dtype=np.float64)
        T[:3, :3] = R
        T[:3, 3] = t
        return T

    def decompose_essential_custom(self, E: np.ndarray, q1: np.ndarray, q2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """
        Декомпозирует Essential-матрицу, выбирает правильную пару (R, t),
        проверяя, сколько точек оказываются перед камерами (cheirality check).
        Также вычисляет относительный масштаб.

        :param E: матрица Essential (3×3)
        :param q1: сопоставленные точки в кадре i-1
        :param q2: сопоставленные точки в кадре i
        :return: правильные (R, t)
        """

        def triangulate_and_score(R, t):
            # Формируем 4×4 трансформацию
            T = self._form_transformation_matrix(R, t)

            # Камера 1: [K | 0], Камера 2: K * [R | t]
            P0 = self.K @ np.hstack((np.eye(3), np.zeros((3, 1))))
            P1 = self.K @ T[:3, :]

            # Триангуляция
            points_4d_hom = cv2.triangulatePoints(P0, P1, q1.T, q2.T)  # 4×N
            points_3d_1 = points_4d_hom[:3, :] / points_4d_hom[3, :]

            # Преобразуем точки в систему второй камеры
            points_4d_cam2 = T @ points_4d_hom
            points_3d_2 = points_4d_cam2[:3, :] / points_4d_cam2[3, :]

            # cheirality: сколько точек с положительным z в обеих камерах
            z1 = points_3d_1[2, :] > 0
            z2 = points_3d_2[2, :] > 0
            valid = z1 & z2
            num_valid = np.sum(valid)

            # относительный масштаб (по длине векторов между соседними точками)
            if np.sum(valid) >= 2:
                dist1 = np.linalg.norm(np.diff(points_3d_1.T[valid], axis=0), axis=1)
                dist2 = np.linalg.norm(np.diff(points_3d_2.T[valid], axis=0), axis=1)
                scale = np.mean(dist1 / dist2)
            else:
                scale = 1.0  # fallback

            return num_valid, scale

        # Получаем 4 возможные пары R, t
        R1, R2, t = cv2.decomposeEssentialMat(E)
        t = np.squeeze(t)
        candidates = [(R1,  t), (R1, -t), (R2,  t), (R2, -t)]

        # Проверяем все и выбираем ту, где больше точек спроецированы вперед
        scores = []
        scales = []
        for R, t_candidate in candidates:
            score, scale = triangulate_and_score(R, t_candidate)
            scores.append(score)
            scales.append(scale)

        best_idx = np.argmax(scores)
        best_R, best_t = candidates[best_idx]
        best_scale = scales[best_idx]

        # Применим относительный масштаб
        t = best_t * best_scale

        # Нормализация и установка фиксированного масштаба
        desired_scale = 0.2
        t = t / np.linalg.norm(t) * desired_scale
        return best_R, t
    
    def estimate_pose_custom(self, pts1: np.ndarray, pts2: np.ndarray) -> Optional[np.ndarray]:
        """
        Оценивает трансформацию (R, t) между двумя изображениями вручную:
        - извлекает матрицу Essential;
        - выполняет собственную декомпозицию;
        - строит 4×4 матрицу преобразования.

        :param pts1: координаты сопоставленных точек на кадре i-1 (Nx2)
        :param pts2: координаты сопоставленных точек на кадре i   (Nx2)
        :return: 4×4 матрица трансформации (или None при неудаче)
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            print("Недостаточно точек для вычисления матрицы Essential.")
            return None

        E, mask = cv2.findEssentialMat(pts1, pts2, self.K, method=cv2.RANSAC, threshold=1.0, prob=0.999)
        if E is None:
            print("Не удалось найти матрицу Essential.")
            return None

        R, t = self.decompose_essential_custom(E, pts1, pts2)
        T = self._form_transformation_matrix(R, np.squeeze(t))
        return T

    @staticmethod
    def filter_pointcloud(points: np.ndarray,
                        z_min: float = 0.1,
                        z_max: float = 10.0,
                        x_range: Optional[Tuple[float, float]] = None,
                        y_range: Optional[Tuple[float, float]] = None,
                        voxel_size: Optional[float] = 0.05,
                        apply_statistical: bool = True,
                        nb_neighbors: int = 20,
                        std_ratio: float = 2.0) -> np.ndarray:
        """
        Фильтрует облако точек на основе геометрических и статистических критериев.

        :param points: Массив 3D-точек формы (N, 3)
        
        :param z_min: Минимально допустимое значение глубины Z. Удаляются точки ближе, чем z_min.
        :param z_max: Максимально допустимая глубина Z. Удаляются слишком дальние точки.
        
        :param x_range: Кортеж (min_x, max_x), ограничивающий координаты X. Если None — не фильтруется.
        :param y_range: Кортеж (min_y, max_y), ограничивающий координаты Y. Если None — не фильтруется.
        
        :param voxel_size: Размер ячейки в voxel grid фильтрации. Если None — voxel-фильтрация не применяется.
                        Используется для упрощения и разреживания точек (например, 0.1 = 5 см).
        
        :param nb_neighbors: Число соседей для оценки плотности (используется при статистической фильтрации).
        :param std_ratio: Порог удаления выбросов — точки, чья средняя дистанция отклоняется на std_ratio,
                        будут удалены.
        
        :return: Отфильтрованное облако точек (M, 3)
        """
        # Фильтрация по глубине и NaN/Inf
        mask = (points[:, 2] > z_min) & (points[:, 2] < z_max)
        mask &= np.isfinite(points).all(axis=1)

        # Фильтрация по X, Y (если заданы)
        if x_range:
            mask &= (points[:, 0] > x_range[0]) & (points[:, 0] < x_range[1])
        if y_range:
            mask &= (points[:, 1] > y_range[0]) & (points[:, 1] < y_range[1])

        # Применение маски
        points = points[mask]
        if len(points) == 0:
            return points

        # Конвертация в формат Open3D
        import open3d as o3d
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points)

        # Voxel Grid — упрощение
        if voxel_size:
            pcd = pcd.voxel_down_sample(voxel_size=voxel_size)

        # Статистическая фильтрация выбросов
        if apply_statistical:
            pcd, _ = pcd.remove_statistical_outlier(nb_neighbors=nb_neighbors, std_ratio=std_ratio)

        return np.asarray(pcd.points)

    def triangulate_points(self,
                        pts1: np.ndarray,
                        pts2: np.ndarray,
                        T1: np.ndarray,
                        T2: np.ndarray) -> Optional[np.ndarray]:
        """
        Выполняет триангуляцию 3D-точек из пары кадров, возвращает точки в мировой системе координат.

        :param pts1: точки в первом изображении (Nx2)
        :param pts2: точки во втором изображении (Nx2)
        :param T1: глобальная поза первой камеры (4×4)
        :param T2: глобальная поза второй камеры (4×4)
        :return: массив 3D-точек в мировой системе (M×3), или None
        """
        if pts1.shape[0] < 8 or pts2.shape[0] < 8:
            return None

        # Построим проекционные матрицы P1, P2 из T1 и T2: P = K * [R|t]
        P1 = self.K @ T1[:3, :]
        P2 = self.K @ T2[:3, :]

        # Триангулируем в однородных координатах
        points_4d = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)  # 4×N
        points_3d = points_4d[:3, :] / points_4d[3, :]             # 3×N → декартовы координаты
        points_3d = points_3d.T                                     # N×3

        # Фильтрация точек
        filtered_points = self.filter_pointcloud(
            points_3d,
            z_min=0.01,
            z_max=100.0,
            voxel_size=None,
            apply_statistical=True
        )

        return filtered_points if len(filtered_points) > 0 else None
    
    def visualize_matches(self, delay_ms: int = 1000):
        num_pairs = len(self.keypoints)
        if num_pairs == 0:
            print("Нет сопоставлений. Сначала вызови run().")
            return

        print("Управление: [A] назад, [D] вперёд, [Q] выход")

        i = 0
        while True:
            kp1, kp2 = self.keypoints[i]
            matches = self.matches[i]
            img1 = self.frames[i]
            img2 = self.frames[i + 1]

            img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
            img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

            vis = cv2.drawMatches(
                img1_color, kp1,
                img2_color, kp2,
                matches, None,
                matchColor=(0, 255, 0),
                singlePointColor=(255, 0, 0),
                flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
            )

            # Отрисовать текущий индекс
            text = f"Frame pair: {i} / {num_pairs - 1}"
            cv2.putText(vis, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

            cv2.imshow("Feature Matches (A/D/Q)", vis)
            key = cv2.waitKey(delay_ms) & 0xFF

            if key == ord('q'):
                print("Выход из визуализации.")
                break
            elif key == ord('d'):
                i = min(i + 1, num_pairs - 1)
            elif key == ord('a'):
                i = max(i - 1, 0)

        cv2.destroyAllWindows()

    def visualize_keypoints_pairwise(self, delay_ms: int = 0):
        """
        Визуализирует пары кадров с нанесёнными ORB-фичами без соединения линиями.

        Управление:
            [A] — назад
            [D] — вперёд
            [Q] — выход
        """
        num_pairs = len(self.keypoints)
        if num_pairs == 0:
            print("Нет сохранённых ключевых точек. Сначала вызови run().")
            return

        print("Управление: [A] назад, [D] вперёд, [Q] выход")

        i = 0
        while True:
            kp1, kp2 = self.keypoints[i]
            img1 = self.frames[i]
            img2 = self.frames[i + 1]

            img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
            img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

            vis1 = cv2.drawKeypoints(img1_color, kp1, None, color=(0, 255, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)
            vis2 = cv2.drawKeypoints(img2_color, kp2, None, color=(255, 0, 0), flags=cv2.DrawMatchesFlags_DRAW_RICH_KEYPOINTS)

            # Объединение изображений
            vis = np.hstack((vis1, vis2))

            # Подпись
            text = f"Frame pair: {i} / {num_pairs - 1}   Points: {len(kp1)} / {len(kp2)}"
            cv2.putText(vis, text, (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)

            cv2.imshow("Keypoints Only (A/D/Q)", vis)
            key = cv2.waitKey(delay_ms) & 0xFF

            if key == ord('q'):
                print("Выход из визуализации.")
                break
            elif key == ord('d'):
                i = min(i + 1, num_pairs - 1)
            elif key == ord('a'):
                i = max(i - 1, 0)

        cv2.destroyAllWindows()
    
    def run(self):
        num_frames = len(self.frames)
        if num_frames < 2:
            raise ValueError("Недостаточно кадров для анализа.")
        
        # Инициализируем начальную глобальную позу (камера в начале на [0, 0, 0])
        # T_total = np.eye(4, dtype=np.float64)
        # self.poses.append(T_total.copy())

        for i in range(num_frames - 1):
            img1 = self.frames[i]
            img2 = self.frames[i + 1]

            kp1, kp2, matches, pts1, pts2 = self.get_matches(img1, img2)
            self.keypoints.append((kp1, kp2))
            self.matches.append(matches)
            print(f"[{i}] Сопоставлено {len(matches)} фич между кадрами {i} и {i+1}")
            
            # if len(matches) < 8:
            #     print(f"[{i}] Пропуск из-за недостаточного количества сопоставлений.")
            #     continue

            # # 2. Оценка движения (R, t) → трансформация T (4x4)
            # T_relative = self.estimate_pose_custom(pts1, pts2)
            # if T_relative is None:
            #     print(f"[{i}] Пропуск из-за ошибки оценки позы.")
            #     continue

            # # 3. Обновляем глобальную позу (умножаем слева!)
            # T_total = T_total @ T_relative
            # T_prev = self.poses[-1]
            # self.poses.append(T_total.copy())
            # print(f"[{i}] Поза обновлена. Камера на позиции: {T_total[:3, 3]}")
            
            # # 4. Триангуляция между позами i и i+1
            # X_world = self.triangulate_points(pts1, pts2, T_prev, T_total)
            # if X_world is not None and len(X_world) > 0:
            #     self.triangulated_points.append(X_world)
            #     print(f"[{i}] Добавлено {len(X_world)} 3D-точек.")

    def visualize_trajectory_xz(self, stride: int = 1, scale: float = 0.3):
        """
        Визуализирует траекторию камеры на плоскости X-Z (вид сверху),
        включая направление взгляда камеры через стрелки.

        :param stride: интервал между стрелками
        :param scale: масштаб стрелок
        """
        if not self.poses:
            print("Нет поз для визуализации.")
            return

        xs = [T[0, 3] for T in self.poses]
        zs = [T[2, 3] for T in self.poses]
        dirs = [(-T[0, 2], -T[2, 2]) for T in self.poses]  # ось Z камеры в мировой системе

        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 6))
        plt.plot(xs, zs, 'k-', label="Траектория")

        for i in range(0, len(xs), stride):
            plt.arrow(xs[i], zs[i],
                    dirs[i][0] * scale, dirs[i][1] * scale,
                    head_width=0.2 * scale, head_length=0.3 * scale,
                    fc='blue', ec='blue')

        plt.xlabel("X")
        plt.ylabel("Z")
        plt.title("Траектория (X-Z, вид сверху)")
        plt.axis('equal')
        plt.grid(True)
        plt.legend()
        plt.show()
        
    def visualize_trajectory_xy(self, stride: int = 1, scale: float = 0.3):
        """
        Визуализирует траекторию камеры на плоскости X-Y (вид сбоку),
        включая направление взгляда камеры через стрелки.

        :param stride: интервал между стрелками
        :param scale: масштаб стрелок
        """
        if not self.poses:
            print("Нет поз для визуализации.")
            return

        xs = [T[0, 3] for T in self.poses]
        ys = [T[1, 3] for T in self.poses]
        dirs = [(-T[0, 2], -T[1, 2]) for T in self.poses]  # ось Z камеры в мировой системе

        import matplotlib.pyplot as plt
        plt.figure(figsize=(10, 6))
        plt.plot(xs, ys, 'k-', label="Траектория")

        for i in range(0, len(xs), stride):
            plt.arrow(xs[i], ys[i],
                    dirs[i][0] * scale, dirs[i][1] * scale,
                    head_width=0.2 * scale, head_length=0.3 * scale,
                    fc='green', ec='green')

        plt.xlabel("X")
        plt.ylabel("Y")
        plt.title("Траектория (X-Y, вид сбоку)")
        plt.axis('equal')
        plt.grid(True)
        plt.legend()
        plt.show()
    
    def visualize_pointcloud(self, every_nth_pose: int = 5):
        """
        Визуализирует объединённое облако точек и траекторию движения камеры в 3D.

        :param every_nth_pose: интервал между визуализируемыми ориентациями камеры
        """
        import open3d as o3d

        # 1. Объединяем все 3D-точки
        if not self.triangulated_points:
            print("Нет 3D-точек для визуализации.")
            return

        all_points = np.vstack(self.triangulated_points)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(all_points)

        # Цвет облака — светло-серый
        gray = np.tile(np.array([[0.7, 0.7, 0.7]]), (len(all_points), 1))
        pcd.colors = o3d.utility.Vector3dVector(gray)

        # 2. Визуализируем путь и оси камеры
        trajectory_frames = []
        for i, T in enumerate(self.poses):
            if i % every_nth_pose != 0:
                continue
            frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3)
            frame.transform(T)  # преобразование в мировую систему
            trajectory_frames.append(frame)

        # 3. Отрисовка сцены
        o3d.visualization.draw_geometries(
            [pcd] + trajectory_frames,
            window_name="3D-реконструкция и путь камеры",
            width=1000,
            height=800,
            zoom=0.6,
            front=[0.0, 0.0, -1.0],
            lookat=[0, 0, 0],
            up=[0.0, -1.0, 0.0]
        )


In [115]:
import cv2
from pathlib import Path

VIDEO_PATH = DATA_PATH_VIDEO / 'hallway.mp4'
CALIBRATION_PATH = DATA_PATH_VIDEO / 'calibration.yaml'
TARGET_FPS = 4.0

if not VIDEO_PATH.exists():
    raise FileNotFoundError(f'Видео не найдено по пути: {VIDEO_PATH.resolve()}')

if not CALIBRATION_PATH.exists():
    raise FileNotFoundError(f'Калибровочный файл не найден: {CALIBRATION_PATH.resolve()}')

params = {
    "target_fps": TARGET_FPS,  # частота обработки кадров
    
    # ORB-параметры
    "nfeatures": 3000,           # [default=500] больше фич — выше плотность покрытия сцены
    "fastThreshold": 10,          # [default=20] ниже порог — чувствительнее к слабым углам
    "edgeThreshold": 10,         # [default=31] позволяет детектировать ближе к краям кадра
    "scaleFactor": 1.2,          # [default=1.2] масштаб между уровнями пирамиды (меньше — медленнее, но точнее)
    "nlevels": 10,               # [default=8] число уровней пирамиды — выше = лучше устойчивость к масштабу

    # Фильтрация матчей по дистанции (Lowe’s ratio test)
    "ratio_thresh": 0.75,         # [default=0.75] строгая фильтрация: оставлять только уверенные совпадения
    "max_match_distance": 60,

    # (опционально можно добавить далее:)
    "flann_table_number": 10,
    "flann_key_size": 14,
    "flann_multi_probe_level": 2,
    "flann_checks": 50
}


vo = VisualOdometry(
    video_path=VIDEO_PATH,
    calibration_file=CALIBRATION_PATH,
    **params
)


Информация о видео:
- Имя файла: hallway.mp4
- Размер кадра: 720 x 1280
- Оригинальный FPS: 30.00
- Общее число кадров: 544
- Длительность видео: 18.13 секунд
- Целевая частота обработки: 4.0 FPS
- Будет обрабатываться каждый 7-й кадр
Обработано кадров: 78 при целевом FPS: 4.0


In [116]:
# from IPython.display import display, clear_output
# from PIL import Image
# import time
# import cv2

# num_frames_to_show = int(TARGET_FPS * 10)
# frames_to_show = vo.frames[:num_frames_to_show]

# for frame in frames_to_show:
#     rgb_frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
#     clear_output(wait=True)
#     display(Image.fromarray(rgb_frame))
#     time.sleep(1 / TARGET_FPS / 2)

In [117]:
vo.clear_data()
vo.run()

[0] Сопоставлено 946 фич между кадрами 0 и 1
[1] Сопоставлено 851 фич между кадрами 1 и 2
[2] Сопоставлено 853 фич между кадрами 2 и 3
[3] Сопоставлено 634 фич между кадрами 3 и 4
[4] Сопоставлено 639 фич между кадрами 4 и 5
[5] Сопоставлено 669 фич между кадрами 5 и 6
[6] Сопоставлено 458 фич между кадрами 6 и 7
[7] Сопоставлено 550 фич между кадрами 7 и 8
[8] Сопоставлено 443 фич между кадрами 8 и 9
[9] Сопоставлено 395 фич между кадрами 9 и 10
[10] Сопоставлено 386 фич между кадрами 10 и 11
[11] Сопоставлено 371 фич между кадрами 11 и 12
[12] Сопоставлено 353 фич между кадрами 12 и 13
[13] Сопоставлено 326 фич между кадрами 13 и 14
[14] Сопоставлено 321 фич между кадрами 14 и 15
[15] Сопоставлено 328 фич между кадрами 15 и 16
[16] Сопоставлено 367 фич между кадрами 16 и 17
[17] Сопоставлено 400 фич между кадрами 17 и 18
[18] Сопоставлено 349 фич между кадрами 18 и 19
[19] Сопоставлено 330 фич между кадрами 19 и 20
[20] Сопоставлено 200 фич между кадрами 20 и 21
[21] Сопоставлено 193

In [118]:
# vo.visualize_keypoints_pairwise()

In [119]:
vo.visualize_matches()

Управление: [A] назад, [D] вперёд, [Q] выход
Выход из визуализации.


In [103]:
vo.visualize_trajectory_xz(stride=2, scale=1)

Нет поз для визуализации.


In [ ]:
# visual_odometry.visualize_trajectory_xy(stride=2, scale=1)

In [ ]:
# visual_odometry.visualize_pointcloud(every_nth_pose=1)

In [ ]:
def visualize_matches_loop(vo: VisualOdometry, delay_ms: int = 100, exit_key: int = 27):
    """
    Визуализирует сопоставленные фичи между кадрами в цикле. Выход — по нажатию клавиши.

    :param vo: объект VisualOdometry с рассчитанными фичами и сопоставлениями
    :param delay_ms: задержка между кадрами (мс)
    :param exit_key: код клавиши для выхода (по умолчанию ESC = 27)
    """
    num_pairs = len(vo.matches)
    if num_pairs == 0:
        print("Нет сопоставлений для визуализации.")
        return

    print(f"Нажмите любую клавишу (код {exit_key}) для выхода из цикла.")
    i = 0
    while True:
        kp1, kp2 = vo.keypoints[i]
        matches = vo.matches[i]
        img1 = vo.frames[i]
        img2 = vo.frames[i + 1]

        # Перевод в BGR для отрисовки
        img1_color = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
        img2_color = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)

        match_vis = cv2.drawMatches(
            img1_color, kp1,
            img2_color, kp2,
            matches, None,
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )

        cv2.imshow("Feature Matches (Press any key to exit)", match_vis)

        key = cv2.waitKey(delay_ms) & 0xFF
        if key != 255:  # любая клавиша (255 — ничего не нажато)
            if key == exit_key:
                print("Выход по клавише.")
            break

        i = (i + 1) % num_pairs  # зацикливание

    cv2.destroyAllWindows()


In [ ]:
visualize_matches_loop(vo, delay_ms=1000)  # например, 300 мс задержка между кадрами